# IP Address Lookup (BMC Discovery)

This notebook reproduces the DisMAL `ipaddr` report for a specific IP address.
It reads connection details from `config.yaml`, queries the appliance for matching devices, discovery access/session results,
and enriches with credential details before exporting a CSV.

## Parameters

In [ ]:
IPADDR = '10.138.168.39'  # e.g., '10.0.0.15'


## Requirements

We use `tideway` from pip (remote), plus `pandas` and `PyYAML`.
Uncomment the following to install in your environment if needed.

In [ ]:
# %pip install -q requests pandas pyyaml

import pandas as pd
import requests
import yaml
from pathlib import Path
from urllib.parse import urljoin
import ipaddress
import json, os
import math
import tideway
import re

## Configuration (from config.yaml)

Reads settings from `../config.yaml` including target, token/token_file,
API version, and SSL verification preference.
Saves the CSV to `../output_<target>/credential_success.csv`.

In [ ]:
from pathlib import Path
import yaml

def load_config_params(
    start: Path,
    appliance_name: str = None,
    appliance_index: int = 0,
) -> dict:
    def _find_repo_root(start: Path) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'config.yaml').exists():
                return p
        return start.parent

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'

    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get('appliances') or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
    token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, 'r') as tf:
            token = tf.read().strip()
    if not token:
        raise ValueError('API token not found in config.yaml (token or token_file)')

    api_version = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
    verify_ssl = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
    }

In [ ]:
def init_appliance(appliance_name: str = "prod"):
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)

    target = params["target"]
    api_version = params["api_version"]
    verify_ssl = params["verify_ssl"]
    output_dir = params["output_dir"]

    print('Base Host     :', target)
    print('API Version   :', api_version)
    print('Verify SSL    :', verify_ssl)
    print('Output folder :', output_dir)

    api_number = api_version.lstrip('v')
    app = tideway.appliance(target, params["token"], api_version=api_number, ssl_verify=verify_ssl)

    try:
        about = app.api_about
        print('Appliance reachable:', about.status_code)
    except Exception as e:
        print('Warning: failed to contact appliance /api/about:', e)

    return {
        "params": params,
        "target": target,
        "app": app,
        "api_version":api_number,
        "output_dir":output_dir,
    }

# Initialise Instances

In [ ]:
print("Initialise Prod:")
twprod = init_appliance("prod")

print("Initialise Dev:")
twdev = init_appliance("dev")

In [ ]:
# bootstrapping variables

token = twprod["params"]["token"]
tokendev = twdev["params"]["token"]
VERIFY_SSL = twprod["params"]["verify_ssl"]
verify_ssl_dev = twdev["params"]["verify_ssl"]
target = twprod["target"]
target_dev = twdev["target"]
api_version = twprod["api_version"]
api_version_dev = twdev["api_version"]
output_dir = twprod["output_dir"]
output_dir_dev = twdev["output_dir"]

BASE_URL = target if ('://' in target) else f'https://{target}'
API_VERSION = api_version

In [ ]:
# Check IP Address has been entered.

if not IPADDR:
    raise ValueError('Please set IPADDR to a valid IP address')

target = twprod

## Helpers (normalization, credentials)

In [ ]:
from typing import Any, Dict, List

def get_credential_map(twcreds_handle) -> Dict[str, Dict[str, Any]]:
    resp = twcreds_handle.get_vault_credentials
    items = to_rows(resp) if hasattr(resp, 'json') else (resp or [])
    if hasattr(resp, 'json'):
        try:
            items = resp.json()
        except Exception:
            items = []
    mapping: Dict[str, Dict[str, Any]] = {}
    if isinstance(items, list):
        for c in items:
            if not isinstance(c, dict):
                continue
            uuid = str(c.get('uuid') or '').split('/')[-1]
            if not uuid:
                continue
            mapping[uuid] = {
                'label': c.get('label'),
                'username': c.get('username')
                            or c.get('snmp.v3.securityname')
                            or c.get('aws.access_key_id')
                            or c.get('azure.application_id'),
                'enabled': c.get('enabled'),
            }
    return mapping


## Run and assemble

In [ ]:
def get_results(instance, qry, columns=['No Results']):
    results = instance['app'].data().search({'query': qry}, format='object', limit=500)
    df = pd.DataFrame(results) if results else pd.DataFrame()
    if df.empty:
        # Provide headers if no rows returned
        df = pd.DataFrame(columns=columns)
    if 'Discovery Instance' not in df.columns:
        df.insert(0, 'Discovery Instance', instance['target'])
    else:
        df['Discovery Instance'] = instance['target']
    return df

def run_and_display(instance, query, columns, head: int = 5):
    """
    Run get_results for an instance, print the target, and display the head of the DataFrame.
    """
    df = get_results(instance, query, columns)
    print(instance['target'])
    display(df.head(head))
    return df

cols = ["Name","OS","Nodekind",]

devices_q = f'''
                    search flags(no_segment) Host, NetworkDevice, Printer, SNMPManagedDevice, StorageDevice, ManagementController
                    where '{IPADDR}' in __all_ip_addrs
                    show
                    name as 'Name',
                    os as 'OS',
                    kind(#) as 'Nodekind'
                    processwith unique()
                '''
#dev_rows = to_rows(twsearch.search({'query': devices_q}, format='object', limit=500))

devices = run_and_display(target, devices_q, cols)

In [ ]:
accesses_q = f'''
                    search DiscoveryAccess where endpoint = '{IPADDR}'
                    show
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.hostname as 'DeviceInfo.hostname',
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.device_type as 'DeviceInfo.device_type',
                    inferred_kind as 'nodekind',
                    (#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_credential
                        or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_slave) as 'DeviceInfo.last_credential',
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_access_method as 'DeviceInfo.last_access_method',
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.method_success as 'DeviceInfo.method_success',
                    'Credential ID Retrieved from DeviceInfo' as 'message'
                    process with unique()
                '''
#acc_rows = to_rows(twsearch.search({'query': accesses_q}, format='object', limit=500))

accesses = run_and_display(target, accesses_q, cols)

In [ ]:
sessions_q = f'''
                    search DiscoveryAccess where endpoint = '{IPADDR}'
                    traverse DiscoveryAccess:Metadata:Detail:SessionResult
                    show
                    session_type as 'session_type',
                    credential as 'credential',
                    success as 'success',
                    message as 'message',
                    kind(#) as 'nodekind'
                    processwith unique()
                '''
#ses_rows = to_rows(twsearch.search({'query': sessions_q}, format='object', limit=5000))

sessions = run_and_display(target, sessions_q, cols)

In [ ]:
dropped_q = '''
                    search DroppedEndpoints
                    show explode endpoints as 'Endpoint'
                '''
#drop_rows = to_rows(twsearch.search({'query': dropped_q}, format='object', limit=5000))

dropped = run_and_display(target, dropped_q, cols)

if IPADDR in dropped['Endpoint'].values:
    print("Note: IP is present in DroppedEndpoints")

In [ ]:
def merge_dis(devices: pd.DataFrame,
              accesses: pd.DataFrame,
              sessions: pd.DataFrame) -> pd.DataFrame:
    # 1) Canonical host key
    dev = devices.copy()
    acc = accesses.copy()
    ses = sessions.copy()

    dev['host'] = dev['Name'].astype(str)
    acc['host'] = acc['DeviceInfo.hostname'].astype(str)

    # 2) Merge devices + accesses on Discovery Instance + host
    merged = pd.merge(
        dev, acc,
        on=['Discovery Instance', 'host'],
        how='outer',
        suffixes=('_dev', '_acc')
    )

    # 3) Coalesce duplicate-ish columns
    def coalesce(a, b):
        return a.where(a.notna(), b)

    # Node kind may appear with different casing
    if 'Nodekind' in merged.columns and 'nodekind' in merged.columns:
        merged['Nodekind'] = coalesce(merged['Nodekind'], merged['nodekind'])
    elif 'nodekind' in merged.columns and 'Nodekind' not in merged.columns:
        merged['Nodekind'] = merged['nodekind']

    # 4) Names: unique list from Name + DeviceInfo.hostname
    def unique_names(row):
        vals = []
        for c in ('Name', 'DeviceInfo.hostname'):
            if c in row and pd.notna(row[c]):
                vals.append(str(row[c]))
        # dedupe, preserve order
        seen = set()
        uniq = [v for v in vals if not (v in seen or seen.add(v))]
        return uniq

    merged['names'] = merged.apply(unique_names, axis=1)

    # 5) Bring in sessions (aggregate per Discovery Instance)
    if not ses.empty:
        ses_agg = ses.groupby('Discovery Instance').agg({
            'session_type':  lambda s: ', '.join(sorted(set(map(str, s.dropna())))),
            'credential':    lambda s: ', '.join(sorted(set(map(str, s.dropna())))),
            'success':       lambda s: any(bool(x) for x in s),
            'message':       lambda s: ', '.join(sorted(set(map(str, s.dropna())))),
            'nodekind':      lambda s: ', '.join(sorted(set(map(str, s.dropna())))),
        }).reset_index().rename(columns={
            'session_type': 'session_types',
            'credential':   'session_credentials',
            'success':      'any_session_success',
            'message':      'session_messages',
            'nodekind':     'session_nodekinds',
        })
        merged = merged.merge(ses_agg, on='Discovery Instance', how='left')

    # 6) Tidy columns: keep the useful ones, drop raw duplicates
    keep_cols = [
        'Discovery Instance', 'host', 'names',
        # from devices
        'OS', 'Nodekind',
        # from accesses (selected)
        'DeviceInfo.device_type', 'DeviceInfo.last_credential',
        'DeviceInfo.last_access_method', 'DeviceInfo.method_success', 'message_acc',
        # from sessions agg
        'session_types', 'session_credentials', 'any_session_success',
        'session_messages', 'session_nodekinds'
    ]
    # Create message_acc if needed (message from accesses side)
    if 'message_acc' not in merged.columns and 'message' in merged.columns:
        # If both sides had 'message', pandas already suffixed; handle that:
        msg_cols = [c for c in merged.columns if c.startswith('message')]
        if 'message_acc' in msg_cols:
            pass
        elif 'message' in merged.columns:
            merged = merged.rename(columns={'message': 'message_acc'})
        elif 'message_acc' not in merged.columns and 'message_dev' in merged.columns:
            merged = merged.rename(columns={'message_dev': 'message_acc'})

    # Filter to existing keepers (in case some are absent)
    keep_cols = [c for c in keep_cols if c in merged.columns]
    out = merged[keep_cols].copy()

    return out

In [ ]:
devices_found = merge_dis(devices, accesses, sessions)

#devices_found = sorted(list({r.get('Name') for r in dev_rows if r.get('Name')} | {r.get('DeviceInfo.hostname') for r in acc_rows if r.get('DeviceInfo.hostname')}))
print(f'IP Address Lookup: {IPADDR}')
display(devices_found)

In [ ]:
# Credential enrichment

# Vault credentials
prodvault = twprod["app"].credentials()

print(twprod['target'])
vault_prod = prodvault.get_vault_credentials.json()
creds = pd.DataFrame(vault_prod)
display(creds.head(10))

In [ ]:
# Map cred label
uuid_to_label = dict(zip(creds['uuid'], creds['label']))
devices_found['credential'] = devices_found['DeviceInfo.last_credential'].map(uuid_to_label)

display(devices_found.head())

## Save CSV

In [ ]:
safe_ip = str(IPADDR).replace(':','_').replace('/','_').replace('.','_')

def save_discovery_run(df: pd.DataFrame, output_dir: Path, filename: str):
    """
    Save a discovery run DataFrame to CSV in the specified output directory.
    """
    output_csv = str(output_dir / f"{filename}.csv")
    df.to_csv(output_csv, index=False)
    print(f"Saved to {output_csv}")

save_discovery_run(devices_found, output_dir, safe_ip)